**Чепкасов Иван 0В32. Практика по получению первичных профессиональных умений и навыков. 2025 г.**

1. Реализовать пайплайн обработки выбранного текста на английском языке, включая всю необходимую предварительную обработку текста (приведение слов к нижнему регистру, удаление стоп-слов, цифр/неалфавитных символов, знаков пунктуации).

In [ ]:
import re
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Пайплайн обработки текста
def preprocess_text(text):
  # Приведение к нижнему регистру
  text = text.lower()

  # Удаление цифр, неалфавитных признаков и знаков пунктуации
  text = re.sub(r'[^a-z\s]', '', text)

  # Токенизация
  tokens = word_tokenize(text)

  # Удаление стоп-слов
  stop_words = set(stopwords.words('english'))
  tokens = [word for word in tokens if word not in stop_words and len(word) > 2]

  return tokens

# Разделение текста на главы
def extract_chapters(text):
    # Формат заголовков: "CHAPTER X: TITLE"
    # Разбиваем текст на главы с сохранением разделителей
    chapters = re.split(r'(^CHAPTER\s+[^:]+:.*$)', text, flags=re.MULTILINE | re.IGNORECASE)

    # Собираем абзацы из непустых частей между заголовками
    paragraphs = []
    for part in chapters:
        # Пропускаем заголовки глав и пустые строки
        if re.match(r'^CHAPTER\s+[^:]+:.*$', part, flags=re.IGNORECASE) or not part.strip():
            continue
        # Удаляем начальные/конечные пробелы и добавляем абзац
        paragraphs.append(part.strip())

    return paragraphs

In [ ]:
with open("document.txt", "r", encoding="utf-8") as f:
  text = f.read()
  f.close()

In [ ]:
# Cписок с токенами каждой главы
chapters = [' '.join(chapter) for chapter in list(map(preprocess_text, extract_chapters(text)))]
print('Токенизированные главы')
for i in range(len(chapters)):
  print(f'Глава {i+1}: ', chapters[i])

Токенизированные главы
Глава 1:  machine learning subfield artificial intelligence focuses building systems learn data three main types supervised learning unsupervised learning reinforcement learning algorithms like linear regression decision trees neural networks form foundation modern applications feature engineering data preprocessing critical steps pipeline
Глава 2:  natural language processing enables computers understand human language key techniques include tokenization stemming lemmatization bagofwords models word embeddings like wordvec represent text numerically applications range sentiment analysis machine translation transformer architectures revolutionized nlp recent years
Глава 3:  dimensionality reduction methods simplify complex datasets principal component analysis pca identifies orthogonal axes maximum variance tsne creates lowdimensional embeddings preserving local structures autoencoders use neural networks nonlinear reduction techniques help visualize highdimensio

2. Разделить текст на главы и в каждой главе отобрать Топ-20 слов с помощью
алгоритма TF-IDF.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Функция для нахождения топ-n слов по TF-IDF в каждой главе
def tf_idf_get_top_words(chapters, top_n=20):
  vectorizer = TfidfVectorizer()
  tf_idf_matrix = vectorizer.fit_transform(chapters)
  feature_names = vectorizer.get_feature_names_out()
  top_words_per_chapter = []
  for i in range(len(chapters)):
      tfidf_scores = zip(feature_names, tf_idf_matrix[i].toarray()[0])
      sorted_words = sorted(tfidf_scores, key=lambda x: x[1], reverse=True)[:top_n]
      top_words_per_chapter.append([word for word, _ in sorted_words])
  return top_words_per_chapter

# Получение топ-20 слов для каждой главы
tf_idf_results = tf_idf_get_top_words(chapters)

# Вывод результатов
print("Топ-20 слов по TF-IDF")
for i in range(len(chapters)):
  print(f"Глава {i+1}: {', '.join(tf_idf_results[i])}")

Топ-20 слов по TF-IDF
Глава 1: learning, data, algorithms, artificial, building, critical, decision, engineering, feature, focuses, form, foundation, intelligence, learn, linear, main, modern, pipeline, preprocessing, regression
Глава 2: language, architectures, bagofwords, computers, enables, key, lemmatization, models, natural, nlp, numerically, processing, range, recent, represent, revolutionized, sentiment, stemming, tokenization, transformer
Глава 3: reduction, autoencoders, axes, complex, component, creates, datasets, help, highdimensional, improve, local, lowdimensional, maximum, nonlinear, orthogonal, pca, performance, preserving, principal, simplify
Глава 4: clustering, clusters, include, data, anomaly, areas, based, builds, centroids, customer, daviesbouldin, dbscan, dendrograms, dense, detection, groups, hierarchical, index, kmeans, metrics
Глава 5: topic, abstract, allocation, automated, coherence, collections, dirichlet, discovers, discovery, document, documents, enable, f

3. Реализовать LDA алгоритм и сравнить результаты с полученными ранее с помощью TF-IDF. Сделать выводы о применимости реализованных подходов.

3.1 Реализация с нуля.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def gibbs_lda(docs, n_topics, alpha, beta, n_iters):

    # Векторизация документов
    cv = CountVectorizer()
    X = cv.fit_transform(docs)
    vocab = cv.get_feature_names_out()
    D, V = X.shape

    # Преобразование документов в списки индексов слов
    docs_idx = [[idx for idx, count in zip(row.indices, row.data) for _ in range(count)]
                for row in X]

    # Инициализация счетчиков
    nw = np.zeros((n_topics, V), dtype=int)     # счетчик тема-слово
    nd = np.zeros((D, n_topics), dtype=int)     # счетчик документ-тема
    nwsum = np.zeros(n_topics, dtype=int)       # число слов в теме
    z = []  # метки тем на словах

    # Случайно первоночально присваеваем метки тем
    for d, doc in enumerate(docs_idx):
        z_d = []
        for w in doc:
            topic = np.random.randint(n_topics)
            z_d.append(topic)
            nw[topic, w] += 1
            nd[d, topic] += 1
            nwsum[topic] += 1
        z.append(z_d)

    alpha_vec = np.ones(n_topics) * alpha
    beta_vec = np.ones(V) * beta

    # Итерации Гиббса
    for _ in range(n_iters):
        for d, doc in enumerate(docs_idx):
            for i, w in enumerate(doc):
                old_topic = z[d][i]
                # Удаление текущей метки
                nw[old_topic, w] -= 1
                nd[d, old_topic] -= 1
                nwsum[old_topic] -= 1

                # Вычисление условных вероятностей
                p = (nd[d] + alpha_vec) * (nw[:, w] + beta_vec[w]) / (nwsum + beta_vec.sum())
                p = p / p.sum()

                # Присваивание новой темы
                new_topic = np.random.choice(n_topics, p=p)
                z[d][i] = new_topic

                # Обновление счетчиков
                nw[new_topic, w] += 1
                nd[d, new_topic] += 1
                nwsum[new_topic] += 1

    # Оценки параметров LDA
    phi = (nw + beta) / (nwsum[:, None] + beta * V)
    theta = (nd + alpha) / (np.sum(nd, axis=1)[:, None] + alpha * n_topics)
    return phi, theta, vocab

Число тем известно заранее в рассматриваемой задаче, однако другие гиперпараметры необходимо подобрать.

In [ ]:
# Метрика - Перплексия
def lda_perplexity(docs, n_topics, alpha, beta, n_iters):

    phi, theta, _ = gibbs_lda(docs, n_topics, alpha, beta, n_iters)

    # Векторизация документов
    cv = CountVectorizer()
    X = cv.fit_transform(docs)

    # Общее число слов во всех тестовых документах
    nd_sum = int(X.sum())

    # Сумма лог‑правдоподобий всех слов
    log_likelihood = 0

    D, V = X.shape
    for d in range(D):
        indices = X[d].nonzero()[1]
        counts  = X[d, indices].toarray().ravel()
        probs = theta[d][:, np.newaxis] * phi[:, indices]
        word_probs = probs.sum(axis=0)
        log_likelihood += np.dot(counts, np.log(word_probs))

    perp = np.exp(- log_likelihood / nd_sum)
    return perp

# Множество значений параметров
param_grid = {'alpha': np.arange(0.005, 0.1, 0.005), 'beta': np.arange(0.025, 0.04, 0.005)}

# Полный перебор
params_values = (0.1, 0.01)
perplexity = lda_perplexity(docs=chapters, n_topics=5, alpha=params_values[0], beta=params_values[1], n_iters=1000)

for alpha in param_grid['alpha']:
  for beta in param_grid['beta']:
    cur_perplexity = lda_perplexity(docs=chapters, n_topics=5, alpha=alpha, beta=beta, n_iters=1000)
    if cur_perplexity < perplexity:
      perplexity = cur_perplexity
      params_values = (round(alpha, 2), round(beta, 2))

print(params_values)

(np.float64(0.01), np.float64(0.04))


In [ ]:
phi, theta, vocab = gibbs_lda(docs=chapters, n_topics=5, alpha=0.01, beta=0.04, n_iters=2000)

In [ ]:
# Находим топ-20 слов для каждой темы.
top20 = {}
for k in range(5):
    top_idx = phi[k].argsort()[-20:][::-1]
    top20[f"Тема {k+1}"] = vocab[top_idx]
df_topics = pd.DataFrame({topic: pd.Series(words) for topic, words in top20.items()})
df_topics.head()

,Тема 1,Тема 2,Тема 3,Тема 4,Тема 5
0,analysis,topic,learning,include,applications
1,reduction,uses,data,clusters,years
2,techniques,topics,like,clustering,word
3,methods,text,language,data,wordvec
4,model,nonnegative,machine,dense,text


In [ ]:
# Для удобства переименуем столбцы
# Соответствие между найденными темами и главами определено визуально
df_topics = df_topics.rename({'Тема 1': 'Глава 3',
                              'Тема 2': 'Глава 5',
                              'Тема 3': 'Глава 1',
                              'Тема 4': 'Глава 4',
                              'Тема 5': 'Глава 2'}, axis=1)

df_topics_gibbs = df_topics.loc[:, [f'Глава {i}' for i in range(1, 6)]]
df_topics_gibbs

,Глава 1,Глава 2,Глава 3,Глава 4,Глава 5
0,learning,applications,analysis,include,topic
1,data,years,reduction,clusters,uses
2,like,word,techniques,clustering,topics
3,language,wordvec,methods,data,text
4,machine,text,model,dense,nonnegative
5,systems,tokenization,human,daviesbouldin,nmf
6,three,transformer,dimensionality,centroids,modeling
7,subfield,supervised,embeddings,customer,collections
8,tsne,translation,themes,using,allocation
9,unsupervised,building,visualize,together,abstract


Уже видно, что ключевые слова фигурируют в результатах работы обоих алгоритмов. Для полноты картины также реализуем LDA при помощи библиотеки scikit-learn. Данная библиотека использует вариационный Байесовский метод для оценки параметров.

3.2 Использование scikit-learn для реализации LDA.

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV

# Векторизация документов
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(chapters)
vocab = vectorizer.get_feature_names_out()

# Множество подбираемых параметров
param_grid = {'doc_topic_prior': np.arange(0.01, 1, 0.04), 'topic_word_prior': np.arange(0.005, 1, 0.005)}

# Метрика - отрицательная Перплексия
def neg_perplexity(estimator, X_data):
    return -estimator.perplexity(X_data)

grid_search = GridSearchCV(
              estimator=LatentDirichletAllocation(n_components=5, learning_method='batch'),
              param_grid=param_grid,
              scoring=neg_perplexity,
              cv=5)

grid_search.fit(X)
print("Лучшие параметры:", grid_search.best_params_)

Лучшие параметры: {'doc_topic_prior': np.float64(0.93), 'topic_word_prior': np.float64(0.9600000000000001)}


In [ ]:
lda = LatentDirichletAllocation(n_components=5,
                                max_iter=10,
                                learning_method='batch',
                                doc_topic_prior=0.93,
                                topic_word_prior=0.96)
lda.fit(X)

LatentDirichletAllocation(doc_topic_prior=0.93, n_components=5,
                          topic_word_prior=0.96)

In [ ]:
def get_top_words(model, feature_names, n_top=20):

    top_words = {}

    for topic_idx, topic in enumerate(model.components_):
        top_idx = topic.argsort()[-n_top:][::-1]
        top_words[f"Тема {topic_idx+1}"] = feature_names[top_idx]

    return pd.DataFrame({topic: pd.Series(words) for topic, words in top_words.items()})

df_topics = get_top_words(lda, vocab, n_top=20)
df_topics.head()

,Тема 1,Тема 2,Тема 3,Тема 4,Тема 5
0,reduction,data,topic,include,learning
1,analysis,analysis,analysis,data,data
2,techniques,applications,human,clusters,machine
3,embeddings,techniques,text,clustering,like
4,identifies,embeddings,evaluation,applications,applications


In [ ]:
# Для удобства переименуем столбцы
# Соответствие между найденными темами и главами определено визуально
df_topics = df_topics.rename({'Тема 1': 'Глава 3',
                              'Тема 2': 'Глава 2',
                              'Тема 3': 'Глава 5',
                              'Тема 4': 'Глава 4',
                              'Тема 5': 'Глава 1'}, axis=1)

df_topics_bayes = df_topics.loc[:, [f'Глава {i}' for i in range(1, 6)]]
df_topics_bayes

,Глава 1,Глава 2,Глава 3,Глава 4,Глава 5
0,learning,data,reduction,include,topic
1,data,analysis,analysis,data,analysis
2,machine,applications,techniques,clusters,human
3,like,techniques,embeddings,clustering,text
4,applications,embeddings,identifies,applications,evaluation
5,networks,human,model,language,dimensionality
6,neural,text,methods,identifies,methods
7,feature,machine,dimensionality,evaluation,model
8,trees,like,neural,dbscan,uses
9,foundation,identifies,networks,spherical,automated


3.3 Сравнение результатов работы TF-IDF и двух версий LDA.

Глава 1

In [ ]:
pd.DataFrame([tf_idf_results[0], df_topics_gibbs['Глава 1'], df_topics_bayes['Глава 1']]).rename({0: 'TF-IDF', 1: 'LDA (Gibbs)', 2: 'LDA (Bayes)'})

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
TF-IDF,learning,data,algorithms,artificial,building,critical,decision,engineering,feature,focuses,form,foundation,intelligence,learn,linear,main,modern,pipeline,preprocessing,regression
LDA (Gibbs),learning,data,like,language,machine,systems,three,subfield,tsne,unsupervised,understand,trees,neural,intelligence,focuses,learn,modern,networks,nlp,models
LDA (Bayes),learning,data,machine,like,applications,networks,neural,feature,trees,foundation,steps,supervised,learn,building,algorithms,linear,systems,subfield,regression,pipeline


Глава 2

In [ ]:
pd.DataFrame([tf_idf_results[1], df_topics_gibbs['Глава 2'], df_topics_bayes['Глава 2']]).rename({0: 'TF-IDF', 1: 'LDA (Gibbs)', 2: 'LDA (Bayes)'})

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
TF-IDF,language,architectures,bagofwords,computers,enables,key,lemmatization,models,natural,nlp,numerically,processing,range,recent,represent,revolutionized,sentiment,stemming,tokenization,transformer
LDA (Gibbs),applications,years,word,wordvec,text,tokenization,transformer,supervised,translation,building,bagofwords,types,regression,key,numerically,main,lemmatization,natural,linear,pipeline
LDA (Bayes),data,analysis,applications,techniques,embeddings,human,text,machine,like,identifies,reduction,evaluation,methods,dimensionality,model,networks,neural,language,include,natural


Глава 3

In [ ]:
pd.DataFrame([tf_idf_results[2], df_topics_gibbs['Глава 3'], df_topics_bayes['Глава 3']]).rename({0: 'TF-IDF', 1: 'LDA (Gibbs)', 2: 'LDA (Bayes)'})

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
TF-IDF,reduction,autoencoders,axes,complex,component,creates,datasets,help,highdimensional,improve,local,lowdimensional,maximum,nonlinear,orthogonal,pca,performance,preserving,principal,simplify
LDA (Gibbs),analysis,reduction,techniques,methods,model,human,dimensionality,embeddings,themes,visualize,use,variance,scale,principal,performance,preserving,mixtures,networks,maximum,nonlinear
LDA (Bayes),reduction,analysis,techniques,embeddings,identifies,model,methods,dimensionality,neural,networks,data,preserving,datasets,local,maximum,help,performance,variance,autoencoders,tsne


Глава 4

In [ ]:
pd.DataFrame([tf_idf_results[3], df_topics_gibbs['Глава 4'], df_topics_bayes['Глава 4']]).rename({0: 'TF-IDF', 1: 'LDA (Gibbs)', 2: 'LDA (Bayes)'})

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
TF-IDF,clustering,clusters,include,data,anomaly,areas,based,builds,centroids,customer,daviesbouldin,dbscan,dendrograms,dense,detection,groups,hierarchical,index,kmeans,metrics
LDA (Gibbs),include,clusters,clustering,data,dense,daviesbouldin,centroids,customer,using,together,identifies,hierarchical,regions,score,segmentation,separated,metrics,nested,partitions,points
LDA (Bayes),include,data,clusters,clustering,applications,language,identifies,evaluation,dbscan,spherical,customer,detection,dendrograms,hierarchical,dense,index,together,groups,silhouette,nested


Глава 5

In [ ]:
pd.DataFrame([tf_idf_results[4], df_topics_gibbs['Глава 5'], df_topics_bayes['Глава 5']]).rename({0: 'TF-IDF', 1: 'LDA (Gibbs)', 2: 'LDA (Bayes)'})

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
TF-IDF,topic,abstract,allocation,automated,coherence,collections,dirichlet,discovers,discovery,document,documents,enable,factorization,generative,interpretation,latent,lda,matrix,mixtures,modeling
LDA (Gibbs),topic,uses,topics,text,nonnegative,nmf,modeling,collections,allocation,abstract,factorization,coherence,enable,document,dirichlet,discovers,evaluation,documents,latent,lda
LDA (Bayes),topic,analysis,human,text,evaluation,dimensionality,methods,model,uses,automated,dirichlet,matrix,represents,generative,coherence,latent,documents,nmf,document,scale


**Вывод**


---


Как мы видим из сопоставления ключевых слов, определенных каждым из алгоритмов, каждый из них показывает себя довольно хорошо в данной задаче. Из общей картины выбивается вторая глава, которая выделяется LDA-алгоритмами не совсем четко. Тем не менее, следует учитывать, что LDA также может быть использован для выделения любого числа скрытых тем при условии, что нам неизвестны темы, которые встречаются в корпусе документов. Также, вероятно, LDA алгоритмы лучше себя покажут на текстах большего объема.

Две реализации LDA, использующие разные методы оценки параметров, показывают схожие результаты в определении тем, но реализация библиотеки scikit-learn, использующая вариационный Байесовский метод, работает быстрее. Наиболее быстро ключевые слова выявляются при помощи TF-IDF.

Таким образом, TF–IDF подойдёт для быстрого выделения ключевых слов внутри отдельного документа, а LDA — для выявления скрытых тем в целом корпусе.
